In [1]:
import nltk
nltk.download("punkt")

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

from nltk.corpus import stopwords
nltk.download('stopwords')

import tensorflow as tf
import numpy as np 
import tflearn
import json
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:472: FutureWarning: Passing (type, 1) or 

In [2]:
from google.colab import files
files.upload()

Saving intents.json to intents (1).json


{'intents.json': b'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "what\'s up"],\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help you?"],\n         "context_set": ""\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye"],\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure", "You are welcome"]\n        },\n        {"tag": "hours",\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\n         "responses": ["We\'re open every day 8am-11pm", "Our hours are 8am-11pm every day"]\n        },\n        {"tag": "location",\n         "patterns":

In [3]:
with open("intents.json") as json_data:
  intents = json.load(json_data)

In [4]:
words = []
classes = []
documents = []

for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent["tag"]))
    if intent["tag"] not in classes:
      classes.append(intent["tag"])

In [5]:
stop_words = set(stopwords.words("english"))
punctuations = ["?", "!", ",", ".", ":"]
words = [stemmer.stem(w.lower()) for w in words if w not in stop_words]
words = [w for w in words if w not in punctuations]
words = [w for w in words if len(w)>1]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [6]:
training = []
output = []
output_empty = [0]*len(classes)

for doc in documents:
  bow = []
  pattern_words = doc[0]
  pattern_words = [stemmer.stem(w.lower()) for w in pattern_words]
  for w in words:
    bow.append(1) if w in pattern_words else bow.append(0)

  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1 

  training.append([bow, output_row])

training = np.array(training)   


In [7]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [8]:
tf.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

model.fit(train_x, train_y, n_epoch=800, batch_size=10, show_metric=True)
model.save('model.tflearn')

Training Step: 3199  | total loss: 0.12885 | time: 0.008s
| Adam | epoch: 800 | loss: 0.12885 - acc: 0.9751 -- iter: 30/32
Training Step: 3200  | total loss: 0.13555 | time: 0.011s
| Adam | epoch: 800 | loss: 0.13555 - acc: 0.9776 -- iter: 32/32
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
def clean_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemmer.stem(w.lower()) for w in sentence_words]
  return sentence_words

In [10]:
def bow(sentence, words):
  sentence_words = clean_sentence(sentence)
  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i] = 1
  return(np.array(bag))      
  

In [16]:
def classify(sentence):
  results = model.predict([bow(sentence, words)])[0]
  results = [[i, r] for i, r in enumerate(results) if r>0.3]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  return return_list

In [21]:
def response(sentence):
  results = classify(sentence)
  if results:
    while results:
      for intent in intents["intents"]:
        if intent["tag"] == results[0][0]:
          print(random.choice(intent["responses"]))
      results.pop(0)